In [2]:

import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


In [3]:
#데이터 정제
import re
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1 # strip()은 문자열 양쪽 끝에 있는 공백을 제거
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2 # \1은 첫번째 그룹을 뜻한다.
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5 
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

In [4]:
# corpus 정제된 문장을 담을 리스트
corpus = []

# preprocess_sentence 함수를 이용해서 정제된 문장을 담아준다.
# 토큰의 개수가 15개를 넘어가는 문장을 제외
# split 합수로 공백을 기준으로 단어를 나누어 준다.
for sentence in raw_corpus:
    if len(sentence) == 0:
        continue
    if len(sentence.split()) >= 13:  # 토큰 15개 이상이면 삭제하라고 되어있는데, 그 뒤에 학습데이터수가 124960개 이상이면 안된다고 해서 다시 줄임
        continue
    corpus.append(preprocess_sentence(sentence))
        
# 정제된 결과를 10개 확인
corpus[:10]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

In [5]:
#토큰화
# 토큰화

import tensorflow as tf
import numpy as np

def tokenize(corpus):
    # 12000단어를 기억할 수 있는 tokenizer를 만들겁니다 (전체 단어의 수 12000개로 제한)
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 12000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )

    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    # fit_on_texts는 리스트 형태로 결과를 반환
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환
    tensor = tokenizer.texts_to_sequences(corpus)   
 
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

print('\n-------------------------\n')
print('텐서의 갯수:', len(tensor))

[[   2   50    5 ...    0    0    0]
 [   2   17 3125 ...    0    0    0]
 [   2   34    7 ...    0    0    0]
 ...
 [   2  263  192 ...    0    0    0]
 [   2  127    5 ...    0    0    0]
 [   2    7   36 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f3319511970>

-------------------------

텐서의 갯수: 158876


In [6]:
#평가 데이터셋 분리
from sklearn.model_selection import train_test_split

src_input = tensor[:, :-1]  # 행 전체, 마지막 열을 제외한 모든 열
tgt_input = tensor[:, 1:]   # 행 전체, 첫번째 열을 제외한 모든 열 (<start>제외)

enc_train, enc_val, dec_train, dec_val = train_test_split(
    src_input, tgt_input,
    random_state=2022,
    test_size = 0.2
)

In [7]:
# encoding, decoding의 약자인듯. 갑자기 변수명이 바뀌어서 헷갈렸다.
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (127100, 32)
Target Train: (127100, 32)


In [8]:
# shuffle에서 쓰인다.
# shuffle 함수는 고정된 버퍼 크기로 데이터를 섞는데, 
# 완전히 랜덤하게 섞기 위해서는 입력된 데이터 크기보다 큰 수를 입력해 주어야 한다.
# 지금은 train 데이터셋 전체 갯수(127100)를 buffer_size로 설정했다.
BUFFER_SIZE = len(enc_train)
# batch_size : 한 번에 읽어올 데이터의 갯수
BATCH_SIZE = 256
# 여기서 정한 steps_per_epoch은 뒤에 사용되지 않았다.
steps_per_epoch = len(enc_train) // BATCH_SIZE

# tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
# 앞에서 num_words = 12000으로 정해주었음
VOCAB_SIZE = tokenizer.num_words + 1

# tf.data.Dataset.from_tensor_slices()를 이용해 corpus 텐서를 tf.data.Dataset객체로 변환
# Premade Estimator를 사용하기 위해서는 feature 데이터와 label 데이터가 함께 전달하여 dataset을 생성해야 한다.
train_dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
# drop_remainder=True는 마지막 남은 데이터를 버린다는 옵션 설정
# 127100 // 256 : 나머지 데이터는 버림
train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder = True)
train_dataset

<BatchDataset shapes: ((256, 32), (256, 32)), types: (tf.int32, tf.int32)>

In [9]:
# validation 데이터셋도 텐서를 tf.data.Dataset 객체로 변환시켜준다.
test_dataset = tf.data.Dataset.from_tensor_slices((enc_val, dec_val)).shuffle(BUFFER_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE, drop_remainder = True)
test_dataset

<BatchDataset shapes: ((256, 32), (256, 32)), types: (tf.int32, tf.int32)>

In [10]:
#인공지능 만들기

class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        # embedding_size : 워드벡터의 차원 수
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [11]:
for src_sample, tgt_sample in train_dataset.take(1):
    break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 32, 12001), dtype=float32, numpy=
array([[[ 1.5552704e-04, -9.5453623e-05, -1.3263618e-04, ...,
         -3.2901797e-05, -2.2219541e-05, -2.1839049e-04],
        [ 2.5679107e-04, -4.4109158e-05, -9.2191258e-05, ...,
         -1.5329766e-04, -2.1413939e-04, -4.0232309e-04],
        [ 4.9953314e-04,  2.3725689e-04, -4.8546094e-04, ...,
         -2.2333862e-04, -2.4159279e-04, -3.8534106e-04],
        ...,
        [ 5.2797832e-03, -2.1343639e-03,  7.7671901e-04, ...,
          1.0734699e-03, -2.5683004e-04,  2.7400132e-05],
        [ 5.2837920e-03, -2.1610572e-03,  8.0967211e-04, ...,
          1.0805696e-03, -2.6492425e-04,  3.4303830e-05],
        [ 5.2861250e-03, -2.1838974e-03,  8.3938532e-04, ...,
          1.0865378e-03, -2.7285051e-04,  4.1585128e-05]],

       [[ 1.5552704e-04, -9.5453623e-05, -1.3263618e-04, ...,
         -3.2901797e-05, -2.2219541e-05, -2.1839049e-04],
        [ 2.3428944e-04, -2.3839963e-04, -4.7958447e-04, ...,
          1.0689471e-04, 

In [12]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [13]:
# optimizer는 loss를 최소화하는 w(가중치)를 찾는 것 (cs231n에 나온 내용)
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

# model.compile(loss, optimizer, metrics)
model.compile(loss=loss, optimizer=optimizer)
model.fit(train_dataset, validation_data = test_dataset, epochs=10)

Epoch 1/10
496/496 [==============================] - 217s 429ms/step - loss: 1.6397 - val_loss: 1.4061
Epoch 2/10
496/496 [==============================] - 223s 450ms/step - loss: 1.3506 - val_loss: 1.3215
Epoch 3/10
496/496 [==============================] - 222s 448ms/step - loss: 1.2762 - val_loss: 1.2710
Epoch 4/10
496/496 [==============================] - 223s 450ms/step - loss: 1.2194 - val_loss: 1.2344
Epoch 5/10
496/496 [==============================] - 223s 450ms/step - loss: 1.1690 - val_loss: 1.2032
Epoch 6/10
496/496 [==============================] - 224s 451ms/step - loss: 1.1229 - val_loss: 1.1782
Epoch 7/10
496/496 [==============================] - 223s 450ms/step - loss: 1.0806 - val_loss: 1.1581
Epoch 8/10
496/496 [==============================] - 223s 450ms/step - loss: 1.0409 - val_loss: 1.1402
Epoch 9/10
496/496 [==============================] - 224s 452ms/step - loss: 1.0037 - val_loss: 1.1250
Epoch 10/10
496/496 [==============================] - 223s 450m

In [14]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]
    
    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다.
    while True:
        predict = model(test_tensor) # 입력받은 문장의 텐서를 입력합니다.
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다.
        
        # 우리 모델이 새롭계 예측한 단어를 입력 문장의 뒤에 붙여줍니다. 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        
        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면 while 루프를 돌면서 다음 단어를 예측해야합니다.
        if predict_word.numpy()[0] == end_token:
            break
        if test_tensor.shape[1] >= max_len:
            break
    
    generated = ""
    
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다.
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "
    
    return generated # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [15]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you <end> '

In [16]:
generate_text(model, tokenizer, init_sentence="<start> if you", max_len=20)

'<start> if you want me to <end> '

In [17]:
generate_text(model, tokenizer, init_sentence="<start> i wonder", max_len=20)

'<start> i wonder if i could <end> '

In [18]:
generate_text(model, tokenizer, init_sentence="<start> i wanted", max_len=20)

'<start> i wanted to be a good time <end> '

In [19]:
generate_text(model, tokenizer, init_sentence="<start> she loves", max_len=20)

'<start> she loves me <end> '

In [20]:
generate_text(model, tokenizer, init_sentence="<start> please give ", max_len=20)

'<start> please give me a sign <end> '

In [21]:
generate_text(model, tokenizer, init_sentence="<start> if you want ", max_len=20)

'<start> if you want me to <end> '

In [22]:
generate_text(model, tokenizer, init_sentence="<start> say ", max_len=20)

'<start> say it all night <end> '